In [1]:
import torch.nn as nn
import torch
import numpy as np

In [2]:
model_prune = torch.load(r"/media/ps/data/train/LQ/task/prune/prune-tp-0119-prune05.pth", map_location="cpu")['struct']
model = torch.load(r"/media/ps/data/train/LQ/task/prune/weights-allbn-orig/model_0022999.pth", map_location="cpu")['struct']


In [22]:
print(model.state_dict().keys())


odict_keys(['backbone.fpn_lateral3.weight', 'backbone.fpn_lateral3.bias', 'backbone.fpn_output3.weight', 'backbone.fpn_output3.bias', 'backbone.fpn_lateral4.weight', 'backbone.fpn_lateral4.bias', 'backbone.fpn_output4.weight', 'backbone.fpn_output4.bias', 'backbone.fpn_lateral5.weight', 'backbone.fpn_lateral5.bias', 'backbone.fpn_output5.weight', 'backbone.fpn_output5.bias', 'backbone.top_block.p6.weight', 'backbone.top_block.p6.bias', 'backbone.top_block.p7.weight', 'backbone.top_block.p7.bias', 'backbone.bottom_up.stem.conv1.weight', 'backbone.bottom_up.stem.norm1.weight', 'backbone.bottom_up.stem.norm1.bias', 'backbone.bottom_up.stem.norm1.running_mean', 'backbone.bottom_up.stem.norm1.running_var', 'backbone.bottom_up.stem.norm1.num_batches_tracked', 'backbone.bottom_up.res2.0.conv1.weight', 'backbone.bottom_up.res2.0.norm1.weight', 'backbone.bottom_up.res2.0.norm1.bias', 'backbone.bottom_up.res2.0.norm1.running_mean', 'backbone.bottom_up.res2.0.norm1.running_var', 'backbone.bottom_

In [ ]:
print(model_prune.state_dict()['backbone.fpn_lateral3.weight'].shape)

In [ ]:
(model_prune.state_dict()['backbone.fpn_lateral3.weight'] == model.state_dict()['backbone.fpn_lateral3.weight']).all()

In [ ]:
model_prune.state_dict()['backbone.bottom_up.stem.norm1.weight']

In [ ]:
model.state_dict()['backbone.bottom_up.stem.norm1.weight']

In [ ]:
for name, p in model_prune.named_parameters():
    print(p.requires_grad)

In [4]:
def unwrap_model(model):
	# loops through all layers of model, including inside modulelists and sequentials
	layers = []
	def unwrap_inside(modules):
		for m in modules.children():
			if isinstance(m,nn.Sequential):
				unwrap_inside(m)
			elif isinstance(m,nn.ModuleList):
				for m2 in m:
					unwrap_inside(m2)
			else:
				layers.append(m)
	unwrap_inside(model)
	return nn.ModuleList(layers)

In [25]:
from detectron2.layers import Conv2d
layers =[]
for name, m in model.named_modules():
    if isinstance(m, (Conv2d, nn.Conv2d)):
        layers.append((name, m))

In [26]:
layers, len(layers)

([('backbone.fpn_lateral3',
   Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1))),
  ('backbone.fpn_output3',
   Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
  ('backbone.fpn_lateral4',
   Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))),
  ('backbone.fpn_output4',
   Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
  ('backbone.fpn_lateral5',
   Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))),
  ('backbone.fpn_output5',
   Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
  ('backbone.top_block.p6',
   Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))),
  ('backbone.top_block.p7',
   Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))),
  ('backbone.bottom_up.stem.conv1',
   Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)),
  ('backbone.bottom_up.res2.0.conv1',
   Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)),
  ('

In [53]:
all = []
for name, m in layers[:10]:
    print(name, m.weight.shape)
    num = torch.sum(torch.square(m.weight),dim=(1,2,3)).view(-1).tolist()
    all.extend(num)
print(len(all),all)
    # print(num)
    # all = []
    # for layer in m.weight:
    #     # print(layer.shape)
    #     element_squared = [e.item()**2 for e in layer.view(-1)]
    #     element_squared = sum(element_squared)
    #     all.append(element_squared)

    # print(len(all), all)



backbone.fpn_lateral3 torch.Size([256, 128, 1, 1])
backbone.fpn_output3 torch.Size([256, 256, 3, 3])
backbone.fpn_lateral4 torch.Size([256, 256, 1, 1])
backbone.fpn_output4 torch.Size([256, 256, 3, 3])
backbone.fpn_lateral5 torch.Size([256, 512, 1, 1])
backbone.fpn_output5 torch.Size([256, 256, 3, 3])
backbone.top_block.p6 torch.Size([256, 256, 3, 3])
backbone.top_block.p7 torch.Size([256, 256, 3, 3])
backbone.bottom_up.stem.conv1 torch.Size([64, 3, 7, 7])
backbone.bottom_up.res2.0.conv1 torch.Size([64, 64, 3, 3])
2176 [0.9857460260391235, 1.1091723442077637, 0.8101006746292114, 0.9870643019676208, 0.896796703338623, 0.8329982161521912, 1.0223848819732666, 1.0172960758209229, 0.9748764038085938, 0.9343035221099854, 0.9191241264343262, 0.9442851543426514, 0.8475511074066162, 1.0968515872955322, 0.9229971766471863, 0.9177657961845398, 0.8629188537597656, 1.0093989372253418, 0.8839079141616821, 0.9854944348335266, 1.0069220066070557, 0.7975203990936279, 0.949311375617981, 0.91318470239639

In [70]:
model.backbone

<generator object Module.named_children at 0x7f634d63cd60>

In [71]:
for name, m in model.modules():
    print(name, m)
    # if isinstance(m, (Conv2d, nn.Conv2d)):
    #     print(name, m)
    #     print(name, dict(model.named_modules())[name])
        
        

TypeError: cannot unpack non-iterable BlendMask object